#### In this demo we will build a machine learning model to classify sms texts as ham or spam

#### SMS Spam Collection Dataset
Source: https://www.kaggle.com/uciml/sms-spam-collection-dataset


In [69]:
import azureml.core
from azureml.core import Workspace, Dataset

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.36.0 to work with mm-aml


In [70]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

In [71]:
data = pd.read_csv('./datasets/spamformodel.csv')
inferecing_data = pd.read_csv('./datasets/spamformodel.csv')
data.head(5)

,labels,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [72]:
data.describe()

,labels,text
count,3000,3000
unique,2,2851
top,ham,"Sorry, I'll call later"
freq,2591,19


In [64]:
# Create a folder
batch_folder = './batch-data'
os.makedirs(batch_folder, exist_ok=True)
print("Folder created!")

Folder created!


In [65]:
# import pandas as pd


# for key in inferecing_data.index.unique():
#     print(key)
#     df = inferecing_data.loc[key].to_frame()
#     print(df.columns)
#     df = df.rename(columns={key: "text"})
#     print(df.columns)
#     df.to_csv('./batch-data/' + '%d.csv' % int(key), header = False, index = False)

In [50]:
# #list files in directory for 
# import os
# path = './batch-data/'
# file_list = os.listdir(path)

In [51]:
# file_list.remove('.ipynb_checkpoints')
# file_list.remove('.amlignore')
# file_list.remove('.amlignore.amltmp')

# files_to_upload = []

# for f in file_list:
#     files_to_upload.append(os.path.join(path, f))

In [52]:
# files_to_upload[0]

'./batch-data/3000.csv'

In [66]:
# # add as registered dataset in AML
# default_ds = ws.get_default_datastore()
# default_ds.upload_files(files=files_to_upload, # Upload the diabetes csv files in /data
#                        target_path='spam-data-inferencing/', # Put it in a folder path in the datastore
#                        overwrite=True, # Replace existing files of the same name
#                        show_progress=True)



In [73]:
#default_ds.upload(src_dir="batch-data", target_path="batch-data", overwrite=True, show_progress=True)

# Register a dataset for the input data
batch_data_set = Dataset.File.from_files(path=(default_ds, 'spam-data-inferencing/'), validate=False)
try:
    batch_data_set = batch_data_set.register(workspace=ws, 
                                             name='spam-batch-data-inference',
                                             description='inference batch data',
                                             create_new_version=True)
except Exception as ex:
    print(ex)

print("Done!")

Done!


In [74]:
default_ds = ws.get_default_datastore()

#Create a tabular dataset from the path on the datastore (this may take a short while)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'spam-data/spamformodel.csv'))

try:
    tab_data_set = tab_data_set.register(workspace=ws, 
                                        name='email_dataset',
                                        description='email spam or ham data',
                                        tags = {'format':'CSV'},
                                        create_new_version=True)
except Exception as ex:
    print(ex)

# Display the first 20 rows as a Pandas dataframe
tab_data_set.take(20).to_pandas_dataframe()

,labels,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [75]:
X = data['text']
Y = data['labels']

In [76]:
count_vectorizer = CountVectorizer()
transformed_vector = count_vectorizer.fit_transform(X)

In [77]:
transformed_vector.shape

(3000, 6245)

In [78]:
#word frequecy
print(transformed_vector[0])

  (0, 2550)	1
  (0, 5784)	1
  (0, 3124)	1
  (0, 4249)	1
  (0, 1641)	1
  (0, 894)	1
  (0, 3977)	1
  (0, 2928)	1
  (0, 1224)	1
  (0, 2604)	1
  (0, 6120)	1
  (0, 3214)	1
  (0, 1222)	1
  (0, 1431)	1
  (0, 5501)	1
  (0, 2581)	1
  (0, 727)	1
  (0, 5955)	1


In [79]:
tfid_transformer = TfidfTransformer() 
tfidf_vector = tfid_transformer.fit_transform(transformed_vector)

In [80]:
#tfidf score per document
print(tfidf_vector[0])

  (0, 6120)	0.2194752986462319
  (0, 5955)	0.19473901783061617
  (0, 5784)	0.22793693965845888
  (0, 5501)	0.15784322212997065
  (0, 4249)	0.2506070193198575
  (0, 3977)	0.16363891047125267
  (0, 3214)	0.28050545509161845
  (0, 3124)	0.31525135382042524
  (0, 2928)	0.11038091843908045
  (0, 2604)	0.18889359811399475
  (0, 2581)	0.15874972350310806
  (0, 2550)	0.1538778038097993
  (0, 1641)	0.26268283838726564
  (0, 1431)	0.2735917909803556
  (0, 1224)	0.2735917909803556
  (0, 1222)	0.2998760486969354
  (0, 894)	0.2542211973750386
  (0, 727)	0.31525135382042524


In [81]:
x_train, x_test, y_train, y_test = train_test_split(tfidf_vector, Y, test_size = 0.2)

In [82]:
def summarize_classification(y_test, y_pred):
    acc = accuracy_score(y_test, y_pred, normalize=True) #how many predictions correct %
    num_acc = accuracy_score(y_test, y_pred, normalize = False)
    prec = precision_score(y_test, y_pred, average = 'weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    
    print('aacuracy count:', num_acc)
    print('accuracy score:', acc)
    print('precision:', prec)
    print('recall:', recall)

In [83]:
clf = GaussianNB().fit(x_train.toarray(), y_train)

In [84]:
y_pred = clf.predict(x_test.toarray())

In [85]:
summarize_classification(y_test, y_pred)

aacuracy count: 532
accuracy score: 0.8866666666666667
precision: 0.9070244737405159
recall: 0.8866666666666667


## Create Training Script

In [87]:
import os
script_folder = os.path.join(os.getcwd(), "train")
print(script_folder)
os.makedirs(script_folder, exist_ok=True)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz-clean/code/Users/memasanz/email-classification/train


In [88]:
%%writefile $script_folder/classifier_training.py

import argparse
from azureml.core import Run

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
import joblib
import os

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt


def summarize_classification(y_test, y_pred, run):
    acc = accuracy_score(y_test, y_pred, normalize=True) #how many predictions correct %
    num_acc = accuracy_score(y_test, y_pred, normalize = False)
    prec = precision_score(y_test, y_pred, average = 'weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    
    run.log('acc count', num_acc)
    run.log('Accuracy', acc)
    run.log('prec', prec)
    run.log('recall', recall)
    
    print('aacuracy count:', num_acc)
    print('accuracy score:', acc)
    print('precision:', prec)
    print('recall:', recall)
    

# def summarize_classification2(model, x_test, y_test, run):
#     y_hat = model.predict(x_test)
#     acc = np.average(y_hat == y_test)
#     print('Accuracy:', acc)
#     run.log('Accuracy', np.float(acc))

    #change labels to 1 and 0's for this to work
    # calculate AUC
#     y_scores = model.predict_proba(x_test)
#     auc = roc_auc_score(y_test,y_scores[:,1])
#     print('AUC: ' + str(auc))
#     run.log('AUC', np.float(auc))

#     # plot ROC curve
#     fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
#     fig = plt.figure(figsize=(6, 4))
#     # Plot the diagonal 50% line
#     plt.plot([0, 1], [0, 1], 'k--')
#     # Plot the FPR and TPR achieved by our model
#     plt.plot(fpr, tpr)
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('ROC Curve')
#     run.log_image(name = "ROC", plot = fig)
#     plt.show()


def getRuntimeArgs():
    # Get script arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
    args = parser.parse_args()

    return args

def model_train(ds_df, run):
    
    X = ds_df['text']
    Y = ds_df['labels']
    #sklearn pipeline
    clf = Pipeline([
                            ('count_vectorizer', CountVectorizer()),
                            ('classifier', LogisticRegression(solver='lbfgs', max_iter=10000))
                        ])
    #output of convectorizer, feed to classifier
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)
    print('type of x_test')
    print(type(x_test))
    model = clf.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    
    print('*************************')
    print('model predictions:')
    print(y_pred)
    summarize_classification(y_test, y_pred, run)

    return model



def main():
    args = getRuntimeArgs()
    
    # Get the experiment run context
    run = Run.get_context()
    
    dataset_dir = './dataset/'
    os.makedirs(dataset_dir, exist_ok=True)
    ws = run.experiment.workspace
    print(ws)
    
    
    print("Loading Data...")
    data = run.input_datasets['training_data'].to_pandas_dataframe()
    
    
    print(data.columns)
    lr = model_train(data, run)
    
    
    # Save the trained model
    model_file = 'email_classifier.pkl'
    joblib.dump(value=lr, filename=model_file)
    run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

    # Complete the run
    run.complete()


    # Register the model
    run.register_model(model_path='outputs/email_classifier.pkl', model_name='email_classifier',
                       tags={'Training context':'spam or ham'})

    #print('Model trained and registered.')
 

if __name__ == "__main__":
    main()

Overwriting /mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz-clean/code/Users/memasanz/email-classification/train/classifier_training.py


In [89]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.exceptions import ComputeTargetException
user = 'mm'
compute_name = user + "-cluster"
print(compute_name)

# checks to see if compute target already exists in workspace, else create it
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
except ComputeTargetException:
    config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D13",
                                                   min_nodes=0, 
                                                   max_nodes=1)

    compute_target = ComputeTarget.create(workspace=ws, name=compute_name, provisioning_configuration=config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=40)

mm-cluster


## Define Environment

In [90]:
%%writefile $script_folder/experiment_env.yml
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow

Overwriting /mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz-clean/code/Users/memasanz/email-classification/train/experiment_env.yml


In [91]:
from azureml.core import Environment

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification("email classification", script_folder + "/experiment_env.yml")

# Let Azure ML manage dependencies
experiment_env.python.user_managed_dependencies = False 

# Print the environment details
print(experiment_env.name, 'defined.')
print(experiment_env.python.conda_dependencies.serialize_to_string())

email classification defined.
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow



In [92]:
import azureml.core.runconfig
from azureml.core import Environment, Experiment
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails

# Get the training dataset
email_training_ds = ws.datasets.get('email_dataset')

# Create a script config
script_config = ScriptRunConfig(source_directory=script_folder,
                                script='classifier_training.py',
                                arguments = [
                                             '--input-data', email_training_ds.as_named_input('training_data')], # Reference to dataset
                                environment=experiment_env) 

# submit the experiment
experiment_name = user + '-email-classification-model-build'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'mm-email-classification-model-build_1642116679_d741cd92',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-01-13T23:32:40.840641Z',
 'endTimeUtc': '2022-01-13T23:33:03.906695Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '2888315b-dfed-49e7-9bb9-0bb65ccc8d76'},
 'inputDatasets': [{'dataset': {'id': '953aae9e-8638-4254-a26c-f81b426874a2'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'training_data', 'mechanism': 'Direct'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'classifier_training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--input-data', 'DatasetConsumptionConfig:training_data'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {'training_data': {'dataLocation': {'dataset': {'id': '953aae9e-8638-4254-a26c-f81b426874a2',
      'name': 'email_dataset',
      'version